In [1]:
import numpy as np
import pandas as pd
import pickle
from mylab.ana.miniscope.context_exposure.Cminiana import MiniAna as MA
import matplotlib.pyplot as plt

In [2]:
file_path = r"C:\Users\Sabri\Desktop\program\Data\miniscope\part6\session2.pkl"

In [3]:
s = MA(file_path)

FUN:: _load_session
FUN:: _load_session
loading C:\Users\Sabri\Desktop\program\Data\miniscope\part6\session2.pkl
loaded
loaded
dict_keys(['ms_ts', 'S_dff', 'sigraw', 'idx_accepted', 'corrected_ms_ts', 'behavevideo', 'behavelog_time', 'behavelog_info', 'behave_track', 'in_context_mask', 'in_context_coords', 'in_lineartrack_mask', 'in_lineartrack_coords', 'all_track_points', 'Trial_Num', 'process', 'aligned_behave2ms'])
'sigraw' is taken as original self.df
'sigraw' is taken as original self.df


In [10]:
s.result["S_dff"].shape

(38531, 135)

In [11]:
s.add_alltrack_placebin_num(according="Body",place_bin_nums=[3,3,20,3,3,3],behavevideo=r"X:\linear_track_behavior\20200820\CDC-test-201033-20200820-132809.mp4")

'place_bin_No' has been there
'place_bin_No' has been there


In [12]:
choice_side = []
if s.result["behavelog_info"]["Left_choice"][0]>s.result["behavelog_info"]["Right_choice"][0]:
    choice_side.append(0)
else:
    choice_side.append(1)
for i in np.diff(s.result["behavelog_info"]["Left_choice"]):
    if i == 0:
        choice_side.append(1)
    else:
        choice_side.append(0)
s.result["behavelog_info"]["choice_side"]=choice_side


In [188]:
dpca_matrix = pd.DataFrame(s.result["S_dff"]).groupby([s.result["Trial_Num"],s.result["place_bin_No"]]).mean()
dpca_matrix = dpca_matrix.drop(index=(-1))
dpca_matrix

context = s.result["behavelog_info"]["Enter_ctx"]
decision = s.result["behavelog_info"]["choice_side"]
ctx0_decision0_trial = s.result["behavelog_info"]["Trial_Num"][(context==0)&(decision==0)]
ctx0_decision1_trial = s.result["behavelog_info"]["Trial_Num"][(context==0)&(decision==1)]
ctx1_decision0_trial = s.result["behavelog_info"]["Trial_Num"][(context==1)&(decision==0)]
ctx1_decision1_trial = s.result["behavelog_info"]["Trial_Num"][(context==1)&(decision==1)]


# 判断以下变量的size
max_Trial_Num = max(ctx0_decision0_trial.shape[0],ctx0_decision1_trial.shape[0]
     ,ctx1_decision0_trial.shape[0] ,ctx1_decision1_trial.shape[0])# 这个Trial_Num应该是每单个（Trial_Num,feature1,featurn2,...）组合的最大值
neurons = dpca_matrix.shape[1]
placebin = len(set(s.result["place_bin_No"]))
context = len(set(context))
decision = len(set(decision))
print(max_Trial_Num,neurons,place_bin,context,decision)

#构建 空的np.array
trialR = np.full((max_Trial_Num,neurons,placebin,context,decision),np.nan)
trialR.shape

for i,trial in enumerate(ctx0_decision0_trial,0):
    temp_trial_matrix = dpca_matrix.loc[trial]
#     print(temp_trial_matrix.shape,trialR[i,:,temp_trial_matrix.index,0,0].shape)
    trialR[i,:,temp_trial_matrix.index,0,0]=temp_trial_matrix
    
for i,trial in enumerate(ctx0_decision1_trial,0):
    temp_trial_matrix = dpca_matrix.loc[trial]
#     print(temp_trial_matrix.shape,trialR[i,:,temp_trial_matrix.index,0,1].shape)
    trialR[i,:,temp_trial_matrix.index,0,1]=temp_trial_matrix
    
for i,trial in enumerate(ctx1_decision0_trial,0):
    temp_trial_matrix = dpca_matrix.loc[trial]
#     print(temp_trial_matrix.shape,trialR[i,:,temp_trial_matrix.index,1,0].shape)
    trialR[i,:,temp_trial_matrix.index,1,0]=temp_trial_matrix
    
for i,trial in enumerate(ctx1_decision1_trial,0):
    temp_trial_matrix = dpca_matrix.loc[trial]
#     print(temp_trial_matrix.shape,trialR[i,:,temp_trial_matrix.index,1,1].shape)
    trialR[i,:,temp_trial_matrix.index,1,1]=temp_trial_matrix

        

0         1         2         3         4    \
Trial_Num place_bin_No                                                     
1         0             0.072900  0.000000  0.485032  0.123376  0.072374   
          3             0.000000  0.066447  0.000000  0.000000  0.023282   
          4             0.000000  1.238794  0.000000  0.000000  0.050180   
          5             0.236633  1.071724  0.138462  0.000000  0.206118   
          6             0.000000  0.198884  1.006155  0.016218  0.000000   
...                          ...       ...       ...       ...       ...   
131       27            0.161131  0.000000  0.064085  0.000000  0.262911   
          28            0.054234  0.000000  0.558062  0.000000  0.151923   
          32            0.000000  0.000000  0.000000  0.000000  0.000000   
          33            0.084775  0.000000  0.406236  0.000000  0.457301   
          34            0.022617  0.045732  0.017306  0.000000  0.141807   

                             5         6         7         8         9    ...  \
Trial_Num place_bin_No                                                    ...   
1         0             0.935841  0.000325  0.689392  0.000000  0.000455  ...   
          3             0.000000  0.000000  0.874794  0.446504  0.163149  ...   
          4             0.306906  0.000000  0.688965  0.869634  0.352051  ...   
          5             0.115824  0.000000  0.121892  0.785719  1.783660  ...   
          6             0.000000  0.000000  0.000000  0.380421  1.391857  ...   
...                          ...       ...       ...       ...       ...  ...   
131       27            0.000000  0.017778  0.000000  0.410041  0.435785  ...   
          28            0.000000  0.000000  0.000000  0.209005  0.225448  ...   
          32            0.000000  0.198371  0.000000  0.636533  1.208670  ...   
          33            0.000000  0.071174  0.000000  0.000000  0.113824  ...   
          34            0.000000  0.000000  0.000000  0.045071  0.295480  ...   

                             125       126       127       128       129  \
Trial_Num place_bin_No                                                     
1         0             0.288501  6.187250  0.000000  0.000000  2.126566   
          3             0.534651  0.000000  0.202768  0.000000  0.000000   
          4             0.146259  0.394927  0.665685  0.000000  0.000000   
          5             0.874126  0.147076  1.117799  0.000000  0.000000   
          6             0.513841  0.019680  0.326854  0.041217  0.000000   
...                          ...       ...       ...       ...       ...   
131       27            0.000000  0.000000  0.215301  0.000000  0.000000   
          28            0.000000  0.000000  0.000000  0.000000  0.026019   
          32            0.000000  0.000000  0.000000  0.000000  0.000000   
          33            0.000000  0.000000  0.000000  0.000000  0.000000   
          34            0.185726  0.152675  0.000000  0.000000  0.000000   

                             130       131        132       133       134  
Trial_Num place_bin_No                                                     
1         0             1.781187  2.707789  10.251923  2.489951  2.973537  
          3             0.000000  0.000000   4.361157  0.756791  0.000000  
          4             0.000000  0.000000   3.131985  0.522323  0.182403  
          5             0.000000  0.000000   0.000000  0.000000  0.000000  
          6             0.000000  0.000000   1.282193  0.002564  0.662130  
...                          ...       ...        ...       ...       ...  
131       27            0.000000  0.000000   0.000000  1.458560  0.000000  
          28            0.000000  0.000000   0.000000  0.530047  0.000000  
          32            0.000000  0.000000   0.000000  0.000000  0.000000  
          33            0.000000  0.000000   0.000000  1.195778  0.000000  
          34            0.000000  0.004272   0.479682  0.241951  0.000000  

[4183 ro

In [295]:
from dPCA import dPCA

In [299]:
dpca = dPCA.dPCA(labels='pcd',regularizer='auto') 
dpca.protect = ['p']

Z = dpca.fit_transform(R,trialR)

You chose to determine the regularization parameter automatically. This can
                    take substantial time and grows linearly with the number of crossvalidation
                    folds. The latter can be set by changing self.n_trials (default = 3). Similarly,
                    use self.protect to set the list of axes that are not supposed to get to get shuffled
                    (e.g. upon splitting the data into test- and training, time-points should always
                    be drawn from the same trial, i.e. self.protect = ['t']). This can significantly
                    speed up the code.
Start optimizing regularization.
Starting trial  1 / 3
Not all protected axis are at the end! While the algorithm will still work, the performance of the shuffling algorithm will substantially decrease due to unavoidable copies.


IndexError: index 2 is out of bounds for axis 2 with size 2

In [194]:
dpca_matrix_mean = np.expand_dims(dpca_matrix,0)

In [301]:
significance_masks = dpca.significance_analysis(X = R,trialX=trialR ,n_shuffles=10, n_splits=10, n_consecutive=10)

Regularization not optimized yet; start optimization now.
Starting trial  1 / 3
Not all protected axis are at the end! While the algorithm will still work, the performance of the shuffling algorithm will substantially decrease due to unavoidable copies.


IndexError: index 2 is out of bounds for axis 2 with size 2

In [176]:
Z['c'].shape

(10, 2, 2, 35)

In [190]:
placebin = np.arange(35)

plt.figure(figsize=(10,4))
#####

for s in range(2):
    plt.plot(placebin,Z['c'][10,0,s])

plt.title('1st time component')


IndexError: index 10 is out of bounds for axis 0 with size 10

<Figure size 720x288 with 0 Axes>

In [154]:
Z["n"].shape

(10, 2, 2, 35)

In [157]:
Z["c"].shape

(10, 2, 2, 35)

In [158]:
Z["d"].shape

(10, 2, 2, 35)

In [159]:
try:
    del s.result["place_bin_No"]
except:
    pass

(10, 2, 2, 35)